In [1]:
import os
import git2net

print(os.path)

#path to copy of virgin db:
sqlite_db_file = '/home/luc/pip/git2net/group_work_1/git2net_tutorial (Kopie).db'


#set variable to whatever folder the to-be-mined git repository is located:
repo_dir = '/home/luc/pip/git2net/ds_gw_1/TLP'

<module 'posixpath' from '/home/luc/pip/git2net/pip_ds/lib/python3.7/posixpath.py'>


In [2]:
#uncomment this block, if mining is wished

#check, if database file is there; remove if yes
#if os.path.exists(sqlite_db_file):
#    os.remove(sqlite_db_file)
#    print('sqlite_db_file removed!')

#mining process: 
#repo_dir: to-be-mined git repository
#sqlite_db_file: to-be-stored values in database

#git2net.mine_git_repo(repo_dir, sqlite_db_file)

#git2net.mining_state_summary(repo_dir, sqlite_db_file)



In [3]:
import pandas as pd
import sqlite3

#connect to database file:
con = sqlite3.connect(sqlite_db_file)
c = con.cursor()


#create a new table:
query0="""
    CREATE TABLE IF NOT EXISTS df2(
    hash TEXT PRIMARY KEY,
    committer_name TEXT,
    committer_date TEXT,
    original_commit_deletion TEXT,
    levenshtein_dist INTEGER);
"""

#select only rows of table "edits", where edit_type='replacement', 
#merge this with table "commits" along the key 'hash',
#group this by unique hashes, taking the sum of the levenshtein dist of same hashes
#finally, insert this into a new table "df2" to avoid key words as column names:
query1="""
    INSERT INTO df2(hash, committer_name, committer_date, original_commit_deletion, levenshtein_dist)
    SELECT hash, committer_name, committer_date, original_commit_deletion, "SUM(levenshtein_dist)"
    FROM
    (
    SELECT 
    hash,
    committer_name,
    committer_date,
    original_commit_deletion,
    SUM(levenshtein_dist)
    FROM(SELECT * FROM edits INNER JOIN commits ON commits.hash = edits.commit_hash WHERE edit_type = 'replacement')
    GROUP BY hash
    );
"""

query1_1="""
    SELECT count(DISTINCT committer_name) FROM df2;
"""

query1_2="""
    SELECT DISTINCT committer_name FROM df2;
"""


#execute the two queries on the database:
c.execute(query0)
c.execute(query1)
c.execute(query1_1)

#store amount of distinct committer names as int...
number_committers = c.fetchone()[0]

#...before executing the next query:
c.execute(query1_2)

#store names of committers in list:
committer_list = c.fetchall()


#save the queries:
con.commit()

#close the database to make it accessible for others:
con.close()




In [4]:
query01="""
    CREATE TABLE IF NOT EXISTS df3(
    hash TEXT PRIMARY KEY,
    committer_name TEXT,
    committer_date TEXT,
    original_commit_deletion TEXT,
    levenshtein_dist INTEGER);
"""

#sort committer_date ascending:

query2="""
    INSERT INTO df3(hash, committer_name, committer_date, original_commit_deletion, levenshtein_dist)
    SELECT *
    FROM
    (
    SELECT * 
    FROM df2 
    ORDER BY committer_date ASC
    );
"""

query3="""
    DROP TABLE df2;
"""

query4="""
    DROP TABLE df3;
"""


#ordering:
#query0, query2_edited, query3
#then:
#query01, query2_edited, query4


con = sqlite3.connect(sqlite_db_file)
c = con.cursor()

c.execute(query01)
c.execute(query2)
c.execute(query3)
#result: df3

print(c.fetchmany(5))

con.commit()
con.close()



[]


In [5]:
def adding_up(df):
    print(df)
    print(df.shape[0])
    if df.shape[0] == 1:
        return df
    for i in range(df.shape[0]):
        print('i=',i)        
        for j in range(i+1, df.shape[0]):
            if df.iloc[i]['A'] < df.iloc[j]['A']:
                print(df.iloc[i]['D'])
                print(df.iloc[j]['D'])
                tmp = df.iloc[i]['D'] + df.iloc[j]['D']
                df.iloc[i]['D'] = tmp
                print(df.iloc[i]['D'])
                print('Es wird gedropped, anschließend Indexreset:\n')
                df = df.drop(df.index[j])        
                df = df.reset_index(drop=True)
                print(df)
                #print(type(df))
                return df

In [6]:
con = sqlite3.connect(sqlite_db_file)

#output df2 as a pandas dataframe
df = pd.read_sql_query("SELECT * FROM df3;", con=con)

#print(df.head(3))
#print(df.shape)
#print(type(df))

In [7]:
#turn Pandas dataframe into numpy array:
df = df.values

#print(type(df))
#print(df.shape)


In [8]:
import numpy as np

def add_column(df):
    #add a full-length '0'-column to the dataframe:
    new_column = np.array(np.zeros(df.shape[0]))
    return np.column_stack((df, new_column))

In [9]:
df = add_column(df)

In [10]:
#make date stamps readible, by turning them into a datetime-object:

from datetime import datetime

def date_to_int(date_str):
    return datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')

In [11]:
print(date_to_int(df[0][2]))

2010-01-22 18:16:44


In [12]:
#calculate amount of time (in seconds) since last commit for every authors commit; save values in separate column

for i in range(df.shape[0]-1):
    author = df[i][1]
    commit_time = df[i][2]
    for j in range(i+1,df.shape[0]):
        author_next = df[j][1]
        commit_time_next = df[j][2]
        if author == author_next:
                df[j][5] = (date_to_int(commit_time_next) - date_to_int(commit_time)).total_seconds()
                break
            


In [13]:
df = add_column(df)
print(df.shape)

(942, 7)


In [14]:
#calculate a measure for productivity:
#levenshtein_dist / time_between_commits = levenshtein_dist per second

for i in range(df.shape[0]):
    if df[i][5] == 0:
        df[i][6] = 0
    else:     
        df[i][6] = df[i][4] / df[i][5]


In [15]:
df = add_column(df)

In [16]:
#find out, if author collaborated or not:
#'1' if his parents hash is another authors commit hash,
#'0' if not

for i in range(df.shape[0]):
    parent_hash = df[i][3]
    orig_author = 'nobody'
    for j in range(i):
        if df[j][0] == parent_hash:
            orig_author = df[j][1]
            #if original hash found, then break:
            break
    df[i][7] = 'no parent_hash'
    #if author collaborated, save '1', if not, save '0':
    if orig_author != df[i][1] and orig_author != 'nobody':
        df[i][7] = 1
    else:
        df[i][7] = 0


In [17]:
#insert as many additional columns into numpy array as there are committer names:
orig_shape_df = df.shape[1]
for i in range(len(committer_list)):
    df = add_column(df)
print(df.shape)

#write dummy variables into the corresponding column:
for i in range(len(committer_list)):
    for j in range(df.shape[0]):
        if df[j][1] == committer_list[i][0]:
            df[j][i + orig_shape_df] = 1

(942, 19)


In [18]:
#delete all the other columns that are not needed for the regression analysis:
df = np.delete(df,0,1)
df = np.delete(df,0,1)
df = np.delete(df,0,1)
df = np.delete(df,0,1)
df = np.delete(df,0,1)
df = np.delete(df,0,1)

In [19]:
#turn numpy array into pandas dataframe:

#this is done manually, perhaps automate this?!
pdf = pd.DataFrame({'productivity_per_second': df[:,0]})
pdf = pdf.assign(collab_bool = df[:,1])
pdf = pdf.assign(thomas_koch = df[:,2])
pdf = pdf.assign(github = df[:,3])
pdf = pdf.assign(sammys_hp = df[:,4])
pdf = pdf.assign(andre_erdmann = df[:,5])
pdf = pdf.assign(aaditya_bagga = df[:,6])
pdf = pdf.assign(timofey_titovets = df[:,7])
pdf = pdf.assign(connor_prussin = df[:,8])
pdf = pdf.assign(tk = df[:,9])
pdf = pdf.assign(kai_heng_feng = df[:,10])
pdf = pdf.assign(maxime_gauduin = df[:,11])
pdf = pdf.assign(qiang_yu= df[:,12])

print(pdf.shape)


(942, 13)


In [20]:
#perform multiple linear regression:

from sklearn.linear_model import LinearRegression

mlr = LinearRegression()

mlr.fit(pdf[['collab_bool', 'thomas_koch', 'github', 'sammys_hp', 'andre_erdmann', 'aaditya_bagga', 'timofey_titovets', 'connor_prussin', 'tk', 'kai_heng_feng', 'maxime_gauduin', 'qiang_yu']], pdf[['productivity_per_second']])

print(mlr.intercept_)
print(mlr.coef_)

[-3.79310801e+12]
[[-1.07474422e+00  3.79310801e+12  3.79310801e+12  3.79310801e+12
   3.79310801e+12  3.79310801e+12  3.79310801e+12  3.79310801e+12
   3.79310801e+12  3.79310801e+12  3.79310801e+12  3.79310801e+12]]


In [21]:
#try a prediction:
#what is the productivity_per_second for thomas_koch, who is not collaborating? :
mlr.predict([[0,1,0,0,0,0,0,0,0,0,0,0]])

array([[0.30371094]])